# Introduction to NLP Fundamentals in Tensorflow

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
2.14.0-dev20230730


In [2]:
import os

if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' already exists, skipping download.")  print("hello world")

# Import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

[INFO] 'helper_functions.py' already exists, skipping download.


We will be using RNN to train textual information

**Basic Structure of RNN**
* Input layer
* text_vectorizer(input)
* embedding(x)
* LSTM -> activation = tanh
* output_Dense -> activation = sigmoid

### Get the text dataset

In [3]:
if not os.path.exists("nlp_getting_started.zip"):
    !wget https://storage.googlqeapis.com/ztm_tf_course/nlp_getting_started.zip
    unzip_data("nlp_getting_started.zip")
else:
  print("[INFO] already exists, skipping download.")

[INFO] already exists, skipping download.


### Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [6]:
#Shuffle to train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head(10)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
5559,7934,rainstorm,NaN,@Calum5SOS you look like you got caught in a r...,0
1765,2538,collision,NaN,my favorite lady came to our volunteer meeting...,1
1817,2611,crashed,NaN,@brianroemmele UX fail of EMV - people want to...,1
6810,9756,tragedy,"Los Angeles, CA",Can't find my ariana grande shirt this is a f...,0
4398,6254,hijacking,"Athens,Greece",The Murderous Story Of AmericaÛªs First Hijac...,1


In [7]:
# test_data
test_df.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [8]:
#How many examples in each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# visualise some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
print(random_index)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  index,text,target = row
  print(f"Target: {target}, {'(real disaster)' if target > 0 else '(not a real disaster)'}")
  print(f"Text:\n{text}\n")
  print("---")

3842
Target: 1, (real disaster)
Text:
CLEARED: COLLISION: #QEW Fort Erie bound approaching Hwy 405 #Niagara.Vehicles removed. ^ag

---
Target: 1, (real disaster)
Text:
'i'm a Gemini' *children screaming buildings burning police sirens in the distance*

---
Target: 0, (not a real disaster)
Text:
#NowPlaying Fitz And The Tantrums - Out Of My League on #Crush #Listen http://t.co/Pwd5L0GLkV #NowPlaying

---
Target: 0, (not a real disaster)
Text:
Detonate (feat. M.O.P.) by Apollo Brown http://t.co/h9FSIaxv3Q

---
Target: 1, (real disaster)
Text:
RT @LivingSafely: #NWS issues Severe #Thunderstorm Warnings for parts of #AR #NC #OK. Expect more trauma cases: http://t.co/FWqfCKNCQW

---


In [10]:
len(train_df_shuffled)

7613

In [11]:
#Splitting the data to create validation -> (train + validation)
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
#Check length
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check few sample
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

1. tokenization -> can get too big as the number of text sample increases
2. Embedding -> richer representation of relationship be between tokens, deeper the realtion between words, more the number/embedding for that token

In [14]:
import tensorflow as tf

# Use the default TextVectorization variables
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                                    standardize="lower_and_strip_punctuation", # how to process text
                                                    split="whitespace", # how to split tokens
                                                    ngrams=None, # create groups of n-words?
                                                    output_mode="int", # how to map tokens to numbers
                                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                    output_mode="int",
                                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
UD: Rescue (Structural Collapse) - Scott Road @ Ypres Road York (14 Trucks)      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[3297,  389,  384,  155, 2715,  366, 4209,  366,  913, 1444, 2641,
           0,    0,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [21]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Like a cyclone imperialism spins across the globe; militarism crushes peoples and sucks their blood like a... http://t.co/n3VbTC6NCa

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02521712, -0.03995385, -0.02787008, ...,  0.02953912,
          0.04548356, -0.0184963 ],
        [-0.04742103, -0.03805054,  0.01006617, ..., -0.02917347,
          0.03854902, -0.01171148],
        [-0.03845384,  0.03355044, -0.02179354, ...,  0.00211239,
          0.01500447, -0.00451385],
        ...,
        [ 0.00103496, -0.00590596, -0.04481291, ..., -0.02390187,
          0.04464345, -0.02162257],
        [ 0.03433524, -0.0230232 , -0.04098833, ..., -0.01836843,
          0.01278043,  0.0391877 ],
        [ 0.04841543, -0.00079918, -0.027134  , ...,  0.01893729,
          0.00502295,  0.00709897]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.02521712, -0.03995385, -0.02787008, -0.03072032, -0.0229486 ,
        0.04970168, -0.04144182, -0.01176939,  0.00726591, -0.04163698,
        0.01985577,  0.0010412 ,  0.03516861,  0.04317958,  0.00764851,
        0.02240009, -0.01546852,  0.01024116,  0.03872153,  0.04595447,
       -0.04427702, -0.00740733, -0.03815686, -0.00969627, -0.04256742,
       -0.00046264,  0.01859811,  0.01433331,  0.00787827, -0.03260972,
       -0.04677233,  0.04007814, -0.00504233,  0.03043962,  0.02347567,
        0.00270803,  0.03976897, -0.01958885,  0.00972883,  0.02695355,
        0.01554685, -0.01288265, -0.03202213,  0.0121213 ,  0.04493102,
       -0.00280536,  0.01657306, -0.04398184, -0.03400229,  0.01618787,
        0.01959412, -0.04563957,  0.00665854,  0.02522229,  0.01925811,
        0.00380262, -0.02588862, -0.04924994, -0.03750138, -0.01190181,
        0.03869111,  0.01330981,  0.03822413,  0.01494822, -0.02149   ,
        0.007397

### Modelling a text dataset
More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data



In [24]:
#model 0
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
from helper_functions import calculate_results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 1: A simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [29]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230815-221316


Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.6100 - accuracy: 0.6916 - val_loss: 0.5365 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4418 - accuracy: 0.8196 - val_loss: 0.4694 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3469 - accuracy: 0.8606 - val_loss: 0.4591 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2853 - accuracy: 0.8920 - val_loss: 0.4641 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2383 - accuracy: 0.9120 - val_loss: 0.4765 - val_accuracy: 0.7861


In [30]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 1ms/step - loss: 0.4765 - accuracy: 0.7861


[0.4764658212661743, 0.7860892415046692]

#### Recurrent Neural Network
* RNN's are useful for sequence data
* The premise of a recurrent neural network is to use the representation of previous input to aid the representation of a later input

### Model 2 : LSTM -> Long short term memory

Input(text) -> Toeknize -> Embedding -> Layers(RNN/Dense) -> Output(label probability)

In [31]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)

model_2_embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)

print(x.shape)

x = tf.keras.layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x) # return vector for whole sequence

print(x.shape)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs)
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_2.summary()

(None, 15, 128)


(None, 64)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                  

In [32]:
# Fit model
model_2_history = model_2.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230815-221324
Epoch 1/5


215/215 [==============================] - 7s 21ms/step - loss: 0.5093 - accuracy: 0.7468 - val_loss: 0.4635 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3270 - accuracy: 0.8697 - val_loss: 0.5139 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2299 - accuracy: 0.9136 - val_loss: 0.5540 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1705 - accuracy: 0.9394 - val_loss: 0.6168 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1245 - accuracy: 0.9537 - val_loss: 0.7639 - val_accuracy: 0.7717


In [33]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.02824819],
       [0.71171445],
       [0.9987287 ],
       [0.03648589],
       [0.00334846],
       [0.99815494],
       [0.7532851 ],
       [0.99843675],
       [0.99913436],
       [0.03084637]], dtype=float32)

In [34]:
# Convert preds to laels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [35]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [36]:
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 77.16535433070865,
 'precision': 0.7744533306963004,
 'recall': 0.7716535433070866,
 'f1': 0.7690403853056418}

#### Model 3: GRU

1. Gated recurrent unit is effective and popular
2. GRU cell has similar features to LSTM cell but has less paramenter

In [37]:
# Build GRU RNN
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.GRU(64, return_sequences=True)(x)
# x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.GRU(64)(x)
# x = tf.keras.layers.Dense(64,activation="relu")(x)
outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs,outputs)

model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences,val_labels))

Epoch 1/5


215/215 [==============================] - 5s 18ms/step - loss: 0.2317 - accuracy: 0.9088 - val_loss: 0.5293 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1607 - accuracy: 0.9419 - val_loss: 0.5962 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1286 - accuracy: 0.9530 - val_loss: 0.7458 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.1100 - accuracy: 0.9606 - val_loss: 0.7495 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0921 - accuracy: 0.9667 - val_loss: 0.7265 - val_accuracy: 0.7887


In [38]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[0.06313486],
       [0.81993896],
       [0.99950814],
       [0.04537889],
       [0.00112807],
       [0.990877  ],
       [0.685639  ],
       [0.999678  ],
       [0.9993538 ],
       [0.6145198 ]], dtype=float32)

In [39]:
# Convert model 3 pred_probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [40]:
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 78.87139107611549,
 'precision': 0.7912533510367916,
 'recall': 0.7887139107611548,
 'f1': 0.7866156427918749}

#### Model4 - Bidirectional RNN

In [41]:
# Trying out on my own
text_vectorizer_exp = tf.keras.layers.TextVectorization(max_tokens=10000,
                                      output_sequence_length=15)
text_vectorizer_exp.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer_exp(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4_exp = tf.keras.Model(inputs,outputs)
model_4_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_4_history_exp = model_4_exp.fit(train_sentences,
                                      train_labels,
                                      validation_data=(val_sentences,val_labels),
                                      epochs=5)

Epoch 1/5
215/215 [==============================] - 7s 22ms/step - loss: 0.5042 - accuracy: 0.7514 - val_loss: 0.4513 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3105 - accuracy: 0.8756 - val_loss: 0.5190 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2067 - accuracy: 0.9200 - val_loss: 0.5619 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1432 - accuracy: 0.9545 - val_loss: 0.6568 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1048 - accuracy: 0.9670 - val_loss: 0.6768 - val_accuracy: 0.7677


Normal RNN's go from left to right

Bi-directional go from right to left as well as left to right

In [42]:
# model 4
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                    output_sequence_length=15)
text_vectorizer.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4 = tf.keras.Model(inputs,outputs)

model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_4_history = model_4.fit(train_sentences,train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 6s 19ms/step - loss: 0.5087 - accuracy: 0.7472 - val_loss: 0.4571 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3124 - accuracy: 0.8714 - val_loss: 0.5173 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2125 - accuracy: 0.9184 - val_loss: 0.5731 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1481 - accuracy: 0.9479 - val_loss: 0.6575 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1071 - accuracy: 0.9634 - val_loss: 0.6487 - val_accuracy: 0.7782


In [43]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[0.02677594],
       [0.83657426],
       [0.99931014],
       [0.12339379],
       [0.0070813 ],
       [0.99716187],
       [0.84193045],
       [0.99968576],
       [0.9995467 ],
       [0.31294748]], dtype=float32)

In [44]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_4_results = calculate_results(val_labels,model_4_preds)
model_4_results

{'accuracy': 77.82152230971128,
 'precision': 0.7791991586469295,
 'recall': 0.7782152230971129,
 'f1': 0.7766208088282468}

#### 1D CNN

In [46]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = tf.keras.layers.Conv1D(32,
                                 5,
                                 activation="relu",
                                 padding="valid")
conv_1d_output = conv_1d(embedding_test)

max_pool = tf.keras.layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [47]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00503846,  0.05494894,  0.07673048, ...,  0.00804908,
         -0.02182053, -0.05185097],
        [ 0.01115147, -0.05215415, -0.04195415, ...,  0.06444334,
         -0.03121651, -0.01923873],
        [-0.05631355, -0.03770373,  0.04916621, ..., -0.01582393,
          0.02511875, -0.01660086],
        ...,
        [ 0.02803942, -0.02742071, -0.00573016, ...,  0.01846335,
          0.01420794,  0.03477152],
        [ 0.02803942, -0.02742071, -0.00573016, ...,  0.01846335,
          0.01420794,  0.03477152],
        [ 0.02803942, -0.02742071, -0.00573016, ...,  0.01846335,
          0.01420794,  0.03477152]]], dtype=float32)>

In [48]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.        , 0.        , 0.04584773, 0.        , 0.14421907,
         0.        , 0.        , 0.        , 0.03124209, 0.        ,
         0.02791934, 0.        , 0.01502951, 0.01838862, 0.02993876,
         0.        , 0.00733377, 0.        , 0.        , 0.        ,
         0.00219617, 0.04659328, 0.        , 0.        , 0.05088214,
         0.        , 0.        , 0.        , 0.03441423, 0.        ,
         0.00203421, 0.        ],
        [0.01750383, 0.04475143, 0.        , 0.        , 0.        ,
         0.02321841, 0.01238304, 0.        , 0.        , 0.01713889,
         0.        , 0.        , 0.13041148, 0.        , 0.0355353 ,
         0.        , 0.09869361, 0.05602964, 0.        , 0.        ,
         0.04940937, 0.        , 0.        , 0.        , 0.        ,
         0.00657529, 0.01086194, 0.0384114 , 0.        , 0.        ,
         0.        , 0.04026869],
        [0.        , 0.00921871, 0.05464577, 0.0462

In [49]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.0447995 , 0.04475143, 0.05464577, 0.04623362, 0.14421907,
        0.04766119, 0.01238304, 0.00061002, 0.03124209, 0.01713889,
        0.02791934, 0.02549994, 0.13041148, 0.10116562, 0.0355353 ,
        0.        , 0.1274805 , 0.05602964, 0.        , 0.03662883,
        0.07600945, 0.04659328, 0.03638304, 0.02705712, 0.05088214,
        0.04203095, 0.04424132, 0.04914358, 0.03441423, 0.03829741,
        0.01447843, 0.04278316]], dtype=float32)>

In [50]:
# Create 1D CNN layer to model sequences
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Conv1D(filters=64,
                           kernel_size=5,
                           padding="valid",
                           activation="relu")(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_5 = tf.keras.Model(inputs,outputs)

model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5_history = model_5.fit(train_sentences,train_labels,
                              validation_data=(val_sentences,val_labels),
                              epochs=5)

Epoch 1/5


215/215 [==============================] - 5s 18ms/step - loss: 0.1512 - accuracy: 0.9482 - val_loss: 0.7520 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1011 - accuracy: 0.9622 - val_loss: 0.8921 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0798 - accuracy: 0.9686 - val_loss: 0.9628 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.0689 - accuracy: 0.9730 - val_loss: 1.0404 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.0602 - accuracy: 0.9749 - val_loss: 1.1145 - val_accuracy: 0.7730


In [51]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 869us/step


array([[2.0414421e-01],
       [6.2309343e-01],
       [9.9995732e-01],
       [5.2368708e-02],
       [3.2611757e-07],
       [9.9801338e-01],
       [9.9478084e-01],
       [9.9988186e-01],
       [9.9999857e-01],
       [6.3455284e-01]], dtype=float32)

In [52]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [53]:
model_5_results = calculate_results(val_labels,model_5_preds)
model_5_results

{'accuracy': 77.29658792650919,
 'precision': 0.77365375135567,
 'recall': 0.7729658792650919,
 'f1': 0.77144803030122}

In [54]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 6 - Tensorflow Hub pretrained models
USE-extractor - universal sentence encoder (USE)

In [55]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed([sample_sentence,
                      "When you can the unvidersal sentence encider on a sentence, it turns into into numbers"])
print(embed_sample[0][:50])

tf.Tensor(
[-0.01157027  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680985  0.05589838 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105888  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209305  0.0267898   0.03011008  0.00321069
 -0.00337971 -0.04787356  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666384  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871508  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758105  0.03327729], shape=(50,), dtype=float32)


In [56]:
embed_sample

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157027,  0.0248591 ,  0.02878048, ..., -0.00186125,
         0.02315824, -0.01485021],
       [ 0.06090042, -0.08857404,  0.01838204, ..., -0.00653699,
         0.03106263,  0.03701009]], dtype=float32)>

Use the above USE"" embedding layer for a normal type of model

In [57]:
# Create a keras layer using the USE"" pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer(hub.load("https://tfhub.dev/google/universal-sentence-encoder/4"),
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [58]:
sentence_encoder_layer

In [59]:
# Create  sequential layer
model_6 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(1,activation="sigmoid")
])

model_6.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_6_history = model_6.fit(train_sentences,train_labels,
                               validation_data=(val_sentences,val_labels),
                               epochs=5)

Epoch 1/5
215/215 [==============================] - 2s 6ms/step - loss: 0.6509 - accuracy: 0.7297 - val_loss: 0.6145 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5828 - accuracy: 0.7901 - val_loss: 0.5643 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5393 - accuracy: 0.7948 - val_loss: 0.5322 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5103 - accuracy: 0.7964 - val_loss: 0.5108 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4901 - accuracy: 0.7999 - val_loss: 0.4963 - val_accuracy: 0.7887


In [60]:
model_6_pred_probs=model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[0.35093772],
       [0.666372  ],
       [0.84024477],
       [0.35460111],
       [0.63537204],
       [0.7464803 ],
       [0.8228136 ],
       [0.8392036 ],
       [0.7441629 ],
       [0.19271317]], dtype=float32)

In [61]:
model_6_preds=tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
model_6_results = calculate_results(val_labels,model_6_preds)
model_6_results

{'accuracy': 78.87139107611549,
 'precision': 0.7896925793067333,
 'recall': 0.7887139107611548,
 'f1': 0.7873013461184764}

In [63]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [64]:
# Challenge, beat the baseline
model_6_exp = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)
])
model_6_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_6_exp_history = model_6_exp.fit(train_sentences,train_labels,
                                      epochs=5,
                                      validation_data=(val_sentences,val_labels))

model_6_exp_pred_probs = model_6_exp.predict(val_sentences)
print(model_6_exp_pred_probs[:10])

model_6_exp_preds = tf.squeeze(tf.round(model_6_exp_pred_probs))
print(model_6_exp_preds[:10])

model_6_exp_results = calculate_results(val_labels, model_6_exp_preds)
print(model_6_exp_results)

Epoch 1/5


215/215 [==============================] - 2s 6ms/step - loss: 0.4801 - accuracy: 0.7927 - val_loss: 0.4379 - val_accuracy: 0.7992
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4008 - accuracy: 0.8212 - val_loss: 0.4267 - val_accuracy: 0.8071
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3806 - accuracy: 0.8362 - val_loss: 0.4275 - val_accuracy: 0.8241
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3610 - accuracy: 0.8432 - val_loss: 0.4269 - val_accuracy: 0.8123
Epoch 5/5
24/24 [==============================] - 0s 4ms/step
[[0.18730474]
 [0.80202144]
 [0.9977754 ]
 [0.20957755]
 [0.6809267 ]
 [0.7560243 ]
 [0.9920861 ]
 [0.9836865 ]
 [0.9826912 ]
 [0.07892736]]
tf.Tensor([0. 1. 1. 0. 1. 1. 1. 1. 1. 0.], shape=(10,), dtype=float32)
{'accuracy': 81.23359580052494, 'precision': 0.8148798668657973, 'recall': 0.8123359580052494, 'f1': 0.810686575717776}
